# Analise exploratória de dados imobiliários com Mapas Interativos

#### Autor: Rodrigo Araujo
https://www.linkedin.com/in/rodrigo-marcel-araujo-oliveira/

A ideia desse estudo é analisar a distribuição de imovéis no território de São Paulo com auxilio de mapas interativos com Python. Para isso, fizemos um Scraping de algumas informações de um site de vendas e locações de imóveis, geramos um csv com qual vamos trabalhar. <br>

Nesse artigo não vamos tratar sobre Web Scraping, caso tenha interesse no assunto, fique ligado pois em breve teremos nossos cursos oline disponível para você.

In [1]:
import folium
from folium.plugins import MarkerCluster

Folium é uma biblioteca que facilita a visualização dos dados que foram manipulados no Python em um mapa interativo de folhetos. Além disso permite a ligação de dados a um mapa para visualizações, além de transmitir visualizações vetoriais / rasterizadas / HTML avançadas como marcadores no mapa. <br>

A biblioteca possui vários conjuntos de peças integradas do OpenStreetMap, Mapbox e Stamen e suporta conjuntos de peças personalizadas com chaves de API Mapbox ou Cloudmade. foliumsuporta sobreposições de imagem, vídeo, GeoJSON e TopoJSON. <br>

Agora vamos apresentar mais uma biblioteca útil para estudos de geolocalização que é a Geocoders, com essa ferramenta é possível achar a latitude e longitude de um determinado endereço. 

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Trading com Dados")
location = geolocator.geocode("Av. Paulista, 967 - Bela Vista, São Paulo - SP")
print(location.address)
print((location.latitude, location.longitude))

Bradesco, 967, Avenida Paulista, Jardim Paulista, São Paulo, Região Imediata de São Paulo, Região Metropolitana de São Paulo, Região Intermediária de São Paulo, São Paulo, Região Sudeste, 01311-100, Brasil
(-23.5655627, -46.6520045)


## Importando dados de imóveis

In [3]:
import json    

data = []
with open(r'source-4-ds-test.json', encoding="utf8") as f:
    for line in f:
        data.append(json.loads(line))

Entendendo a estrutura dos dados

In [4]:
data[0]

{'address': {'city': 'São Paulo',
  'country': 'BR',
  'district': '',
  'geoLocation': {'location': {'lat': -23.557225, 'lon': -46.662765},
   'precision': 'GEOMETRIC_CENTER'},
  'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Centro>Consolacao',
  'neighborhood': 'Consolação',
  'state': 'São Paulo',
  'street': 'Rua Bela Cintra',
  'streetNumber': '',
  'unitNumber': '',
  'zipCode': '01415000',
  'zone': 'Centro'},
 'bathrooms': 1,
 'bedrooms': 1,
 'createdAt': '2015-10-20T20:52:41Z',
 'description': 'Apartamentos de 1 dormitório na Rua Bela Cintra. Próximos ao centro, ao metrô Paulista e à Av. Paulista.\r\n\r\nAgende sua visita!',
 'id': '89224365f8',
 'images': ['https://s3-sa-east-1.amazonaws.com/vr.images.sp.admin/f87a57aa-9a22-425c-a3b4-086a0fd7c969',
  'https://s3-sa-east-1.amazonaws.com/vr.images.sp.admin/8ae90420-6b40-4afc-baf5-a286e8fd5c04',
  'https://s3-sa-east-1.amazonaws.com/vr.images.sp.admin/7a11138b-c306-4267-885a-ce885797a31b',
  'https://s3-sa-east-1.amazonaws.com/vr.i

A estrutura dos dados estão no formato json, isso dificulda a manipulação dos dados para nossa finalidade nesse estudo, para isso vamos transformar os dados em um dataframe 

In [5]:
from pandas.io.json import json_normalize
df = json_normalize(data)
df.head()

,address.city,address.country,address.district,address.geoLocation.location.lat,address.geoLocation.location.lon,address.geoLocation.precision,address.locationId,address.neighborhood,address.state,address.street,...,pricingInfos.rentalTotalPrice,pricingInfos.yearlyIptu,publicationType,publisherId,suites,title,totalAreas,unitTypes,updatedAt,usableAreas
0,São Paulo,BR,,-23.557225,-46.662765,GEOMETRIC_CENTER,BR>Sao Paulo>NULL>Sao Paulo>Centro>Consolacao,Consolação,São Paulo,Rua Bela Cintra,...,NaN,NaN,STANDARD,967d57ce20,0.0,Apartamento Bela Cintra,47.0,APARTMENT,2018-11-08T15:02:53.953Z,47.0
1,São Paulo,BR,,-23.592852,-46.581879,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Leste>Quinta ...,Quinta da Paineira,São Paulo,Rua Bruno Cavalcanti Feder,...,NaN,0.0,STANDARD,bddebf057a,0.0,"Apartamento residencial à venda, Quinta da Pai...",55.0,APARTMENT,2018-11-08T16:10:49.374Z,55.0
2,São Paulo,,,-23.493609,-46.638456,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Norte>Santa T...,Chora Menino,São Paulo,Rua Copacabana,...,NaN,NaN,STANDARD,d7190e8f4c,1.0,"Apartamento em Santa Terezinha - São Paulo, SP",NaN,APARTMENT,2019-02-12T18:29:26.933Z,92.0
3,São Paulo,,,-23.607981,-46.686180,GEOMETRIC_CENTER,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Brooklin,Brooklin Paulista,São Paulo,Rua Guararapes,...,NaN,368.0,STANDARD,316d75f06f,3.0,Apartamento reformado com varanda gourmet no B...,145.0,APARTMENT,2019-02-24T23:34:29.306Z,145.0
4,São Paulo,,,-23.540604,-46.715088,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Alto de...,Alto de Pinheiros,São Paulo,Rua Pio XI,...,NaN,79.0,STANDARD,295eddde1a,1.0,"Apartamento Residencial à venda, Alto de Pinhe...",76.0,APARTMENT,2019-02-23T00:33:46.237Z,76.0


Excelente, agora temos nossos dados no formato que podemos manusear fácilmente os dados. Suponha que queriamos filtrar somente um bairro específico, ou seja, estou interessado nos imóveis do bairro do Butãnta.

In [6]:
dados_filtrados = df.loc[(df['address.neighborhood']=='Butantã')]
dados_filtrados.head()

,address.city,address.country,address.district,address.geoLocation.location.lat,address.geoLocation.location.lon,address.geoLocation.precision,address.locationId,address.neighborhood,address.state,address.street,...,pricingInfos.rentalTotalPrice,pricingInfos.yearlyIptu,publicationType,publisherId,suites,title,totalAreas,unitTypes,updatedAt,usableAreas
40,São Paulo,,,-23.587339,-46.754017,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Butanta,Butantã,São Paulo,Rua Luísa Crapsi Orsi,...,NaN,0.0,STANDARD,581abc24ae,1.0,"Apartamento residencial à venda, Butantã, São...",65.0,APARTMENT,2019-02-25T06:20:12.700Z,65.0
176,São Paulo,,,-23.563487,-46.721009,APPROXIMATE,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Butanta,Butantã,São Paulo,,...,NaN,0.0,STANDARD,e4faf6b597,0.0,Lindo ! Mairare!!!,NaN,APARTMENT,2019-02-14T02:03:51.662Z,136.0
515,São Paulo,,,-23.585236,-46.777273,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Butanta,Butantã,São Paulo,Rodovia Raposo Tavares,...,NaN,0.0,STANDARD,9975992a34,2.0,Apartamento a venda no Butantã!!Cond Mairare!!!!,200.0,APARTMENT,2018-10-15T18:57:51.020Z,106.0
527,São Paulo,,,-23.578414,-46.730165,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Butanta,Butantã,São Paulo,Rua José Álvares Maciel,...,NaN,30.0,STANDARD,0e4d98bf63,0.0,"Apartamento 3 dormitórios residencial à venda,...",NaN,APARTMENT,2019-02-24T23:14:34.271Z,73.0
1276,São Paulo,,,-23.545715,-46.749176,GEOMETRIC_CENTER,BR>Sao Paulo>NULL>Sao Paulo>Zona Oeste>Butanta,Butantã,São Paulo,Rua Parnamirim,...,NaN,0.0,STANDARD,c0de129662,1.0,"Lindo apartamento com 03 dormitórios , na regi...",82.0,APARTMENT,2018-11-08T15:54:18.029Z,82.0


Agora nosso interesse esta visualizar geograficamente esses imóveis em um mapa. Podemos fazer isso da seguinte forma:

In [7]:
from folium.plugins import MarkerCluster
mapa = folium.Map(location=[-16.1237611, -59.9219642],zoom_start=4)

MarkerCluster = MarkerCluster().add_to(mapa)

for i in range(len(dados_filtrados)):
    folium.Marker(
        location=[dados_filtrados.iloc[i,3], dados_filtrados.iloc[i,4]],
    ).add_to(MarkerCluster)

mapa

A função "folium.Map()" criará um objeto fazendo referência onde estará localizado o mapa. Já a "MarkerCluster()" adicionará Clusters ao mapa, ou seja, os pontos geográficos de interesse. A função "folium.Marker()" contém diversas informações, é nela que definimos as latitudes e longitudes, etc. Abaixo vamos explorar um pouco mais essa função.

### Adicionando a quantidade de quartos do imóvel

In [8]:
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[-23.533773, -46.625290],zoom_start=11)

MarkerCluster = MarkerCluster().add_to(mapa)


for i in range(len(dados_filtrados)):
    folium.Marker(
        location=[dados_filtrados.iloc[i,3], dados_filtrados.iloc[i,4]],
        
        popup = ('Quantidade de quartos: ' + str(dados_filtrados.iloc[i,14]).capitalize(),  
                          
                ),
    ).add_to(MarkerCluster)
    
mapa

Quando clicar nos clusters é possível identificar a quantidade de quartos que há no imóvel. 

## Mapa de Calor

Agora suponha estamos interessados em saber a concentração geográfica dos imóveis, a venda ou para locação, na região do Butantã, Brooklin Paulista e Consolação, vamos explorar esses dados com base nos mapas de calor.  

In [9]:
dados_heatmap = df.loc[(df['address.neighborhood']=='Butantã') | (df['address.neighborhood']=='Brooklin Paulista') | (df['address.neighborhood']=='Consolação')]

In [10]:
from folium.plugins import HeatMap

lat = dados_heatmap.iloc[:,3].tolist()
lng = dados_heatmap.iloc[:,4].tolist()
m = folium.Map(
    location=[-23.533773, -46.625290],
    tiles='cartodbdark_matter',
    zoom_start=10
)
HeatMap(list(zip(lat, lng))).add_to(m)
m

Para montar o mapa utilizamos a função "HeatMap()" e passamos como parâmetro uma lista de tuplas criadas com o zip(). <br>
Vimos que é possível identificar a concentração dos imóveis em determinadas regiões dos bairros escolhidos. Podemos vilualizar o mesmos dados mudando o layout do mapa apenas mudando a variavel "tiles" da seguinte forma:

In [11]:
from folium.plugins import HeatMap

lat = dados_heatmap.iloc[:,3].tolist()
lng = dados_heatmap.iloc[:,4].tolist()
mapa_heat = folium.Map(
    location=[-23.533773, -46.625290],
    tiles='stamentoner',
    zoom_start=10
)
HeatMap(list(zip(lat, lng))).add_to(mapa_heat)
mapa_heat

In [239]:
mapa_heat.save('mapa_heat_imoveis_sampa.html') # Salvando o mapa

A Leaflet e também o Folium, usam o OpenStreetMap para montar seus mapas. Os mapas da OpenStreetMap são desenvolvidos por uma comunidade voluntária de mapeadores que contribuem e mantêm atualizados os dados sobre estradas, trilhos, cafés, estações ferroviárias etc. <br>

Conseguimos ver que as ferramentas que utilizamos são muito poderosas e podemos fazer diversas coisas com elas, a ideia desse artigo foi aprensentar algumas técnicas aplicadas ao mercado imobiliário. <br>

No próximo post vamos discutir sobre tendências do mercado imobiliário. 

#### Referências

https://python-visualization.github.io/folium/ <br>
https://www.openstreetmap.org/about <br>
https://programminghistorian.org/en/lessons/mapping-with-python-leaflet